<a href="https://colab.research.google.com/github/brunokiyoshi/machine_learning/blob/master/churn_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Churn Modelling


In [44]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [45]:
tf.__version__

'2.3.0'

In [46]:
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Churn modelling/Churn_Modelling.csv")
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [47]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [48]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [49]:
# Encode categorical data (country and gender) to labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# First for gender, for all rows, which is on column 3 (index = 2)
X[:, 2] = le.fit_transform(X[:, 2])
X


array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [50]:
# Then encode geography using One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [51]:
# Splitting the dataset into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [52]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train[0])
print(X_test[0])

[-1.01460667 -0.5698444   1.74309049  0.16958176 -1.09168714 -0.46460796
  0.00666099 -1.21571749  0.8095029   0.64259497 -1.03227043  1.10643166]
[-1.01460667  1.75486502 -0.57369368 -0.55204276 -1.09168714 -0.36890377
  1.04473698  0.8793029  -0.92159124  0.64259497  0.9687384   1.61085707]


In [53]:
# Initializing the ANN
ann = tf.keras.models.Sequential()

# Add input layer and first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [54]:
# Add second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [55]:
# Add output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # only one output

In [56]:
# Compiling
# adam = stochastic gradient descent
# binary_crossentropy - always used in binary classification
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
# Training
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 770us/step - loss: 0.5040 - accuracy: 0.7958
Epoch 2/100
250/250 [==============================] - 0s 775us/step - loss: 0.4411 - accuracy: 0.7961
Epoch 3/100
250/250 [==============================] - 0s 842us/step - loss: 0.4214 - accuracy: 0.8124
Epoch 4/100
250/250 [==============================] - 0s 771us/step - loss: 0.4123 - accuracy: 0.8166
Epoch 5/100
250/250 [==============================] - 0s 781us/step - loss: 0.4070 - accuracy: 0.8191
Epoch 6/100
250/250 [==============================] - 0s 846us/step - loss: 0.4031 - accuracy: 0.8205
Epoch 7/100
250/250 [==============================] - 0s 806us/step - loss: 0.3998 - accuracy: 0.8225
Epoch 8/100
250/250 [==============================] - 0s 776us/step - loss: 0.3967 - accuracy: 0.8226
Epoch 9/100
250/250 [==============================] - 0s 801us/step - loss: 0.3937 - accuracy: 0.8240
Epoch 10/100
250/250 [==============================] - 0s 769us/step - l

In [59]:
'''
Geography: French ([1, 0, 0])
Credit score: 600
Gender: Male (1)
Age: 40
Tenure: 3 years
Balance 60K
Products: 2
Credit card: yes
Active member: yes
Salary: 50K
^ is this costumer likely to leave the bank?
'''
# Any input of the predict method should be a 2D array
customer_data = [1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]
customers_to_evaluate = [customer_data]
customers_to_evaluate_scaled = sc.transform(customers_to_evaluate)
print(ann.predict(customers_to_evaluate_scaled) > 0.5)

[[False]]


In [65]:
predictions = ann.predict(X_test)
predictions = (predictions > 0.5)
# print(predictions)
# print(y_test)
print(np.concatenate((np.reshape(predictions, (len(predictions), 1)), np.reshape(y_test, (len(y_test),1))), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, predictions)
print(cm)
accuracy_score(y_test, predictions)

[[1490  105]
 [ 173  232]]


0.861